In [176]:
import numpy as np
import math
import latexify

In [177]:
KERNEL_SIZE = 7
SIGMA = 15

In [178]:
@latexify.function(use_math_symbols=True)
def LoG(x, y, sigma):
    return ((- 1 / (np.pi * sigma**4) ) * (1 - (x**2 + y**2) / (2 * sigma**2) ) ) * np.exp(- (x**2 + y**2) / (2 * sigma**2))
    
def get_kernel(sigma : float, size : int) -> np.array:
    
    ker = np.empty((size, size), dtype=float)
    center_cell = size//2
    
    for ((x, y) , _) in np.ndenumerate(ker):
        ker[x , y] = LoG(x - center_cell , y - center_cell , sigma)
    
    return ker

LoG

In [179]:
ker = get_kernel(SIGMA, KERNEL_SIZE)
ker

array([[-5.79941977e-06, -5.93208997e-06, -6.01276548e-06,
        -6.03983782e-06, -6.01276548e-06, -5.93208997e-06,
        -5.79941977e-06],
       [-5.93208997e-06, -6.06700085e-06, -6.14903683e-06,
        -6.17656535e-06, -6.14903683e-06, -6.06700085e-06,
        -5.93208997e-06],
       [-6.01276548e-06, -6.14903683e-06, -6.23189882e-06,
        -6.25970432e-06, -6.23189882e-06, -6.14903683e-06,
        -6.01276548e-06],
       [-6.03983782e-06, -6.17656535e-06, -6.25970432e-06,
        -6.28760269e-06, -6.25970432e-06, -6.17656535e-06,
        -6.03983782e-06],
       [-6.01276548e-06, -6.14903683e-06, -6.23189882e-06,
        -6.25970432e-06, -6.23189882e-06, -6.14903683e-06,
        -6.01276548e-06],
       [-5.93208997e-06, -6.06700085e-06, -6.14903683e-06,
        -6.17656535e-06, -6.14903683e-06, -6.06700085e-06,
        -5.93208997e-06],
       [-5.79941977e-06, -5.93208997e-06, -6.01276548e-06,
        -6.03983782e-06, -6.01276548e-06, -5.93208997e-06,
        -5.7994197

In [180]:
import cv2
  
image = cv2.imread('OIP.jpeg')

image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  
cv2.imwrite("gray.jpeg", image)

True

In [181]:
original_rows, original_cols = image.shape
print(original_rows, original_cols)

padding = KERNEL_SIZE//2

top_left = np.zeros(shape=(padding, padding))
top_center = np.zeros(shape=(padding, original_cols))
top_right = np.zeros(shape=(padding, padding))
left = np.zeros(shape=(original_rows, padding))
right= np.zeros(shape=(original_rows, padding))
bottom_left = np.zeros(shape=(padding, padding))
bottom_center=np.zeros(shape=(padding, original_cols))
bottom_right=np.zeros(shape=(padding, padding))

padded_image =np.block([[top_left,      top_center,     top_right], 
                        [left,          image,          right],
                        [bottom_left,   bottom_center,  bottom_right]]
)

padded_image.shape

296 474


(302, 480)

In [182]:
convolved_image = np.zeros(shape=image.shape)

In [183]:
for i in range(convolved_image.shape[0]):
    for j in range(convolved_image.shape[1]):
        
        convolved_image[i , j] = np.sum(np.multiply(
            padded_image[i : i + padding*2 + 1 , j : j + padding*2 + 1 ],
            ker 
        ))

convolved_image = np.interp(
    convolved_image, 
    (convolved_image.min(), convolved_image.max()),
    (1, 254)
)

In [184]:
convolved_image

array([[233.52847678, 228.20891508, 223.11744705, ..., 173.83949584,
        187.79238809, 202.36841651],
       [228.58640291, 221.85993069, 215.57159387, ..., 152.146296  ,
        169.63298797, 187.67009911],
       [220.26779573, 212.64365536, 205.19674959, ..., 131.91211328,
        152.48454806, 173.38700315],
       ...,
       [252.93683592, 252.46544468, 252.02460295, ..., 147.96575664,
        165.66094251, 183.47998792],
       [253.34219752, 253.13367698, 252.80546211, ..., 156.07658499,
        172.4205743 , 188.97378904],
       [253.79536563, 253.69968768, 253.44059214, ..., 176.49638442,
        189.15896035, 202.01626158]])

In [185]:
cv2.imwrite("out.jpeg", convolved_image)

True

In [186]:
radius = int (np.sqrt(2) * SIGMA)
print(radius)

rows, columns = image.shape
print(image.shape)

local_maxima = []



for i in range(radius, rows - radius): # check 
    for j in range(radius, columns - radius):
        if image[i, j] == np.amax(image[i - radius : i + radius, j - radius : j + radius ] ):
            local_maxima.append( (i,j))


print(len(local_maxima))

21
(296, 474)
44


In [187]:
local_maxima.sort(key=lambda x: x[1], reverse=True)
local_maxima

[(243, 448),
 (108, 445),
 (184, 441),
 (34, 433),
 (103, 414),
 (199, 412),
 (60, 409),
 (199, 404),
 (136, 402),
 (164, 383),
 (94, 363),
 (236, 360),
 (52, 356),
 (163, 351),
 (216, 332),
 (160, 305),
 (207, 293),
 (90, 284),
 (60, 271),
 (64, 271),
 (219, 263),
 (165, 236),
 (167, 235),
 (229, 210),
 (230, 210),
 (81, 208),
 (274, 190),
 (130, 180),
 (174, 176),
 (212, 158),
 (173, 146),
 (118, 135),
 (33, 130),
 (210, 112),
 (80, 103),
 (118, 89),
 (143, 89),
 (269, 86),
 (56, 79),
 (186, 77),
 (78, 74),
 (32, 64),
 (128, 47),
 (208, 40)]

In [188]:
#rgb_image = cv2.imread("OIP.jpeg")

for (i, j) in local_maxima: 
    cv2.circle(convolved_image, (i, j), radius, (255, 0, 0), thickness=1, lineType=2)

cv2.imwrite("circled.jpeg", convolved_image)


True